**Загрузка библиотек и датасета**

In [ ]:
!pip install gensim scikit-learn

In [ ]:
!pip install numpy==1.23.5

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from gensim.utils import simple_preprocess

In [ ]:
!pip install datasets

In [ ]:
import datasets

In [ ]:
dataset_news = datasets.load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
import random

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short_news = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1

random.shuffle(dataset_short_news)
dataset_news = {'train': dataset_short_news[:6400], 'test': dataset_short_news[6400:]}

In [ ]:
news_X = []
for i in range(len(dataset_news['train'])):
  news_X.append(dataset_news['train'][i]['news'])
for i in range(len(dataset_news['test'])):
  news_X.append(dataset_news['test'][i]['news'])

In [ ]:
news_y = []
for i in range(len(dataset_news['train'])):
  news_y.append(dataset_news['train'][i]['label'])
for i in range(len(dataset_news['test'])):
  news_y.append(dataset_news['test'][i]['label'])

**Токенезация текста**

In [ ]:
token_doc = [simple_preprocess(i) for i in news_X]

In [ ]:
# словарь и Bag of Words
dictionary = corpora.Dictionary(token_doc)
bow_corpus = [dictionary.doc2bow(doc) for doc in token_doc]

In [ ]:
print(f"Всего документов: {len(bow_corpus)}")
print(f"Размер словаря: {len(dictionary)}")

Всего документов: 8000
Размер словаря: 19369


**Проведение эксперимента**

In [ ]:
alpha = ['auto', 2, 1, 0.1, 0.01]
beta = ['auto', 2, 1, 0.1, 0.01]

In [ ]:
num_topics = 10
passes = 15

In [ ]:
for a in alpha:
  for e in beta:
    lda_model = models.LdaModel(
    bow_corpus,
    num_topics=num_topics,
    id2word=dictionary,
    alpha=a,
    eta=e,
    passes=passes,
    random_state=42
    )
    print(f"alpha = {a}, eta = {e}")
    for topic in lda_model.print_topics(num_words=10):
        print(topic)
    print()

alpha = auto, eta = auto
(0, '0.024*"gt" + 0.024*"lt" + 0.022*"the" + 0.022*"to" + 0.019*"google" + 0.018*"its" + 0.016*"of" + 0.015*"on" + 0.014*"com" + 0.013*"and"')
(1, '0.021*"israeli" + 0.018*"court" + 0.017*"sharon" + 0.014*"file" + 0.013*"west" + 0.013*"bank" + 0.012*"gaza" + 0.012*"sharing" + 0.010*"homes" + 0.010*"funds"')
(2, '0.017*"on" + 0.016*"in" + 0.015*"bush" + 0.014*"at" + 0.014*"said" + 0.011*"and" + 0.010*"kerry" + 0.009*"korea" + 0.009*"people" + 0.009*"minister"')
(3, '0.042*"in" + 0.029*"to" + 0.027*"the" + 0.026*"of" + 0.016*"and" + 0.016*"iraq" + 0.015*"on" + 0.015*"najaf" + 0.011*"reuters" + 0.011*"al"')
(4, '0.020*"president" + 0.014*"chavez" + 0.013*"venezuela" + 0.012*"to" + 0.010*"on" + 0.009*"russian" + 0.009*"vote" + 0.009*"referendum" + 0.009*"hugo" + 0.008*"recall"')
(5, '0.077*"the" + 0.029*"to" + 0.028*"in" + 0.027*"of" + 0.019*"and" + 0.015*"for" + 0.012*"on" + 0.009*"at" + 0.009*"ap" + 0.007*"with"')
(6, '0.047*"oil" + 0.035*"prices" + 0.030*"on" + 